In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold,train_test_split
from sklearn import svm
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler

import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score


In [ ]:
#Enter the mRMR filter part,x should be updated according to the filtered columns
!git clone https://github.com/jundongl/scikit-feature.git
%cd scikit-feature/
!python setup.py install
%cd ..
!pip install pyswarms
!wget https://github.com/aishwarya25252/tmp/raw/main/newFilterWrappers.zip
!unzip -o newFilterWrappers.zip
!mv newFilterWrappers/colon_microarray.csv colon_microarray.csv
import newFilterWrappers as nf
def mrmr(x,y,c):
    filter=nf.Multivariate(x,y)
    feature=filter.MRMR()
    x1=x[feature].iloc[:,-c:]
    print(x1)
    return x1
    

fatal: destination path 'scikit-feature' already exists and is not an empty directory.
/content/scikit-feature
running install
running build
running build_py
running install_lib
running install_egg_info
Removing /usr/local/lib/python3.7/dist-packages/skfeature-1.0.0.egg-info
Writing /usr/local/lib/python3.7/dist-packages/skfeature-1.0.0.egg-info
/content
--2021-06-23 17:39:35--  https://github.com/aishwarya25252/tmp/raw/main/newFilterWrappers.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aishwarya25252/tmp/main/newFilterWrappers.zip [following]
--2021-06-23 17:39:35--  https://raw.githubusercontent.com/aishwarya25252/tmp/main/newFilterWrappers.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.g

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# dataset=pd.read_csv('colon_microarray.csv')
# dataset=pd.read_csv('mrmrLeukemia.csv')
# dataset.head()

In [ ]:
dataset.drop(axis=1,columns=[dataset.columns[0]],inplace=True)

NameError: ignored

In [ ]:
l=LabelEncoder()
y=l.fit_transform(dataset.iloc[:,-1]) #the class column
y.shape

In [ ]:
x=dataset.iloc[: , :-1]
x.shape

In [ ]:
x=pd.DataFrame(MinMaxScaler().fit_transform(x))

In [ ]:
x.head()

In [ ]:
r=x.shape[0]#62
c=x.shape[1]#2000
numberofsolutions=15

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import RandomForestClassifier

In [ ]:
import random
from sklearn.model_selection import KFold 

In [ ]:
r=x.shape[0]#62
c=x.shape[1]#2000
print(r, ", ", c)

NameError: ignored

In [ ]:
class Solution:
        
    def __init__(self):
        t=[]
        for i in range(c):
          t.append(random.random())
        self.solution=t

      #  self.solution=np.random.uniform(0,1,size=c)
        self.fitness=self.loocv()
        self.limit=0
        #print("Initializing solution:\n")
        #self.printsol()


    def tupletolist(self,arr):
       # print("tupletolist ",arr,"\n")
        l=[]
        for i in arr:
          l.append(i)
        return(l)

    #converts a cotinuous valued array to binary
    def converttobinary(self):  
      x=[]
      for i in self.solution:
            x.append(int(np.round((abs(i%2)))%2))
      return(x)
    
    def ClassificationAccuracy(self):
      model= svm.SVC(kernel='poly')
      arr=np.nonzero(self.converttobinary())
     
      ret=self.tupletolist(arr[0])
      x_train,x_test,y_train,y_test=train_test_split(x.iloc[:,ret],y,test_size=0.3)
      m=model.fit(x_train,y_train)
      predicted_y = model.predict(x_test)
     # print("accuracy_score(y_test,predicted_y): ",accuracy_score(y_test,predicted_y))
      return(accuracy_score(y_test,predicted_y))

      

    #return average kfold crossvalidation score
    def fitnessfunc(self,k=10):
        model= svm.SVC(kernel='poly')
        ret=self.tupletolist(np.nonzero(self.converttobinary())[0])
        score=cross_val_score(model, x.iloc[:,ret], y,cv=10,scoring='accuracy')
       # print("10 fold cv scores ",score,"\n")
        return(sum(score)/k)
    
    # return loocv score
    def loocv(self):
     
      cv=LeaveOneOut()
      model=svm.SVC(kernel='poly')
      ret=self.tupletolist(np.nonzero(self.converttobinary())[0])
      if ret<=0:
        return(0)
      scores=cross_val_score(model, x.iloc[:,ret], y, scoring='accuracy', cv=cv)
   #   print("CV score ",scores,"\n")
      return(sum(scores)/len(scores))

    def printsol(self):
      print(self.tupletolist(np.nonzero(self.converttobinary())[0]))



In [ ]:
  class ABC:
    
    def __init__(self,numberofsolutions):
        self.pop=[]
        for i in range(numberofsolutions):
              self.pop.append(Solution())

    def printsol(self):
      print(self.tupletolist(np.nonzero(self.converttobinary())[0]))

    def bestsol(self):
        f=[]
        m=-1
        bestsolution=0
        ctr=0
        indx=0
        for i in self.pop:
            t=i.fitness
            f.append(t)
            if m<t:
                indx=ctr
                m=t
                bestsolution=i
            ctr+=1
        return(bestsolution) #return best solution index in pop

    #update and generate new solution newS
    def update(self,i):  
    #    print("updating   ")
        r=np.random.randint(0,len(self.pop)) #choosing random neighbour
        while(r==i):
            r=np.random.randint(0,len(self.pop)) # random neighbour index in pop 
        newS=[] #new solution vector
        rn=np.random.uniform(-1,1) 
       # for j in range(c):
       #   newS.append(self.pop[i].solution[j]+rn*(self.pop[i].solution[j]-self.pop[i].solution[r]))
       
        newS=self.pop[i].solution
        rj=np.random.randint(0,c) #random dimension to update
        newS[rj]+=rn*(newS[rj]-self.pop[r].solution[rj])
        newSolution=Solution()
        newSolution.solution=newS
        
        newSolution.fitness=newSolution.loocv()

       # print(np.nonzero(newSolution.converttobinary())[0])
        return newSolution 

    def employed(self):
         # print("Employed bee ")
          for i in range(len(self.pop)):
            c=self.update(i)
            if self.pop[i].fitness<c.fitness:
                self.pop[i]=c
            else:
                self.pop[i].limit+=1

    def onlooker(self):
       # print("Onlookerbee Phase\n")
        p=[]  #possibility of selecting a particular solution
        s=0
        for i in self.pop: #calculate total fitness
            s+=i.fitness
    
        for i in self.pop:  #calculate probability of each
            p.append(i.fitness/s)
        
        select=[]
        
        noOfOnlooker=numberofsolutions
        count=0     
        i=0
        while count<=noOfOnlooker:
          if np.random.rand()<p[i]:#solution selected 
            select.append(i)
            newS=self.update(i)
          
            child=self.crossover()
            if child.fitness>self.pop[i].fitness:
              self.pop[i]=child
            else:
              self.pop[i].limit+=1

          count+=1
          i+=1
          if i==len(self.pop):
            i=0  
        
 
     
    def scout(self,maxlimit=5):

      c=0
      lmaxlimit=[] #stores indices of solutions that have exceeded maxlimit
      for i in range(len(self.pop)):
        if self.pop[i].limit>maxlimit:
          lmaxlimit.append(i)
      if len(lmaxlimit)>=2:
        #first scout bee
        lenl=len(lmaxlimit)
        r=np.random.randint(0,lenl)
        scoutindex=lmaxlimit.pop(r) 
        self.pop[scoutindex]=Solution()

        #second scout bee
            
        mutate=self.mutation(queen)
        self.pop[scoutindex]=mutate
    
        
    #Apply uniform crossover
    def crossover(self):
      # print("from crossover function")
      selecting_neighbor=random.randint(0, len(self.pop)-1)

      parentqueen=queen.solution #continuous value array like a chromosome
      parentneighbour=obj.pop[selecting_neighbor].solution #continous value array like a chromosome

      f11=[]
      f22=[]
      for j in range(len(parentqueen)):
        rn=random.uniform(0,1)
        if rn>0.6:
          f11.append(parentqueen[j])
          f22.append(parentneighbour[j])
        else:
          f11.append(parentneighbour[j])
          f22.append(parentqueen[j])

      f1=Solution()
      f1.solution=f11
      f1.fitness=f1.loocv() #calc fitness

      f2=Solution()
      f2.solution=f22
      f2.fitness=f2.loocv() #calc fitness

      if f1.fitness>f2.fitness:
        newSolution=f1
      else:
        newSolution=f2
      
     # print("Child: ", np.nonzero(winner.converttobinary())[0])
      return(newSolution)    
    
    def mutation(self,i):
        mprVal=0.01
        selectedsol = queen.solution
        #for all the genes in the selectedsol
        for j in range(len(selectedsol)):
          #random no.
          rn=random.uniform(0,1)
          if rn<mprVal:
            rij=random.uniform(-1,1)

            randBidx=random.randint(0, len(self.pop)-1)
            randB=(self.update(randBidx)).solution
            # randB=obj.pop[selecting_neighbor].solution 

            # child=self.update(i)
            selectedsol[j]=queen.solution[j]+(rij*(randB[j]-queen.solution[j]))
        
        f1=Solution()
        f1.solution=selectedsol
        f1.fitness=f1.loocv() #calc fitness
        return(f1)


In [ ]:
maxIter=20 #100
queen=[]
numberofsolutions=15

In [ ]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest, SelectPercentile

In [ ]:
  #mrmr code
  X=mrmr(x,y,c)

KeyboardInterrupt: ignored

In [ ]:
def DATA_PROCESS(url_link="https://raw.githubusercontent.com/NSECResearchCD-SLB/PEERJ_MFSAC_EC/main/CANCER%20GENE%20EXPRESSION%20DATASETS/leukemia.csv"):
    DF=pd.read_csv(url_link,sep="\t",header=None)
    X=DF.iloc[:,:-1]
    Y=DF.iloc[:,-1]
    X=pd.DataFrame(MinMaxScaler().fit_transform(X))
    return X,Y
(x,y)=DATA_PROCESS()

NameError: ignored

In [ ]:
  #mrmr code
  model= svm.SVC(kernel='poly')
  x=pd.read_csv('mrmrLeukemia.csv',sep="\t")
  no=50
  x1=x.iloc[:,:no]
  cv=LeaveOneOut()
  
  scores=cross_val_score(model, x1, y, scoring='accuracy', cv=cv)
  a=sum(scores)/len(scores)
  max=0
  count=0
  while count<10:
    #add 50 more features in x1
    no+=50
    x1=x.iloc[:,:no]
    #evaluation using loocv
    scores=cross_val_score(model, x1, y, scoring='accuracy', cv=cv)
    a=sum(scores)/len(scores)
    if a>=max:
        max=a
    else:
      x1=x.iloc[:,:no-10]
      break
    count+=1
 


In [ ]:
x=x1

In [ ]:
x.shape

(72, 550)

In [ ]:
x=pd.read_csv('mrmrLeukemia.csv',sep="\t")

FileNotFoundError: ignored

In [ ]:
if __name__ == "__main__":
    
    c=x.shape[1]
    r=x.shape[0]
    print(c, " ", r)
      
    #queen=[]
    obj=ABC(80) # population initialised 

   
    Mlimit=5
    for i in range(maxIter):
        print("abc iter: ", i)
        # print("Going to employed bee iter: ",i)
        obj.employed()
        queen=obj.bestsol()

        
        # print("Going to onlooker bee phase iter: ",i)
        obj.onlooker()
        queen=obj.bestsol()
      #  print(" Queen in Onlooker Phase Iter ",i," is ",obj.pop[queen].printsol())
        
        # print("Going to ScoutBee iter ",i)
        obj.scout(Mlimit)
        # queen.append(obj.pop[obj.bestsol()])
        queen=obj.bestsol()
       # print(" Queen in Scout Phase Iter ",i," is ",obj.pop[queen].printsol())

550   72
abc iter:  0
abc iter:  1
abc iter:  2
abc iter:  3
abc iter:  4
abc iter:  5
abc iter:  6
abc iter:  7
abc iter:  8
abc iter:  9
abc iter:  10
abc iter:  11
abc iter:  12
abc iter:  13
abc iter:  14
abc iter:  15
abc iter:  16
abc iter:  17
abc iter:  18
abc iter:  19


In [ ]:

def func(i):
  return(obj.pop[i].fitnessloocv())


# for i in queen:
#   print(obj.pop[i].fitnessloocv(), "\n")

In [ ]:
bestsolution=queen
maxcf=0
for _ in range(15):
  xx=bestsolution.ClassificationAccuracy()
  if maxcf < xx:
    maxcf=xx
print(maxcf)
print(len(np.nonzero(bestsolution.converttobinary())[0]))

1.0
256


In [ ]:
bestsolution.fitnessfunc()

0.9464285714285714

In [ ]:
bestsolution.loocv()

0.9861111111111112

In [ ]:
a=0
m=0
for i in range(100):
  t=bestsolution.ClassificationAccuracy()
  a+=t
  if t>m:
    m=t


print(a/100)
print(m)

0.9836363636363633
1.0
